In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Env

In [2]:
import os
import random
from tqdm import tqdm
import torch
import torchvision
import numpy as np

from PIL import Image

from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torchvision import transforms
import torch.optim as optim

In [ ]:
from modules.solver import Solver

In [4]:
device = torch.device("cuda:2")

# Dataset

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

# Task 1

### Train ResNet

In [ ]:
net = torchvision.models.resnet50(pretrained=False, num_classes=10)

test_tensor = torch.randn(4, 3, 32, 32)
print(net(test_tensor).shape)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
solver = Solver(net, optimizer, criterion, device=device)

In [ ]:
solver.train(epochs=40, data_loader=trainloader, log=True, save_per_epoch=10, val_loader=testloader)

In [ ]:
solver.test(testloader)

### pretrain

In [ ]:
net = torchvision.models.resnet50(pretrained=True)

In [ ]:
pretrained_resnet = nn.Sequential(*list(net.children())[:-1])

# 添加新的全连接层
num_classes = 10
projection =  nn.Sequential(
    nn.Flatten(),
    nn.Linear(2048, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 256), 
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Linear(256, 10),
)

net_with_projection = nn.Sequential(
    pretrained_resnet,
    projection
)
# for param in pretrained_resnet.parameters():
#     param.requires_grad = False

net_with_projection.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_with_projection.parameters(), lr=0.001, momentum=0.9)

In [33]:
solver = Solver(net_with_projection, optimizer, criterion, device=device)

In [ ]:
epoch = 10
log = True
solver.train(epoch, trainloader, log,  save_per_epoch=5, val_loader=testloader)

In [26]:
solver.load_checkpoints("/data/shengkuan/CS172/cs172-assignment1-2023fall-skkk256/checkpoints/resnet34_fintune4_replace.pth")

In [ ]:
solver.test(testloader)

In [ ]:
from modules.ResNet import resnet50

# Untrained model
my_model = resnet50()

# Pretrained model
my_model = resnet50(pretrained=True, checkpoints_path="/data/shengkuan/CS172/cs172-assignment1-2023fall-skkk256/checkpoints/resnet50.pt")
my_model.eval() # for evaluation

In [ ]:
solver = Solver(my_model, optimizer, criterion, device=device)
solver.test(trainloader)

# Task 2

In [ ]:
import cv2

from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader

from modules.dataloader import SHHA_loader
from modules.crowd_counting import data_collate, draw_and_save
from modules.utils import get_density_map_gaussian
from modules.ResNet import MCNN
from modules.ResNet import UNet, resnet50
import torch.optim as optim


import os
import random
from tqdm import tqdm
import torch
import torchvision
import numpy as np

from PIL import Image

from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torchvision import transforms
import torch.optim as optim

In [11]:
device = torch.device("cuda:2")

In [12]:
data_path = "/data/shengkuan/CS172/cs172-assignment1-2023fall-skkk256/data/ShanghaiTech_Crowd_Counting_Dataset/part_B_final"
output_size = 512
num_workers = 4
batch_size = 8

train_dataset = SHHA_loader(data_path, "train", output_size)
test_dataset = SHHA_loader(data_path, "test", output_size)
train_loader = DataLoader(
    train_dataset, batch_size, True,
    num_workers=num_workers, pin_memory=True, drop_last=True, collate_fn=data_collate)
test_loader = DataLoader(
    test_dataset, batch_size, False,
    num_workers=num_workers, pin_memory=True, drop_last=False, collate_fn=data_collate)
    

In [ ]:
dataiter = iter(test_loader)
image, gt = next(dataiter)
print(image[0].min(), image[0].max())

In [ ]:
# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

fg, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))


# get some random training images
dataiter = iter(test_loader)
for inputs in dataiter:
    images, gt = inputs
    # # show images
    # imshow(torchvision.utils.make_grid(images))
    # print(gt[0].shape)
    # # print labels
    # # print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
    # draw_and_save(images, gt, "/data/shengkuan/CS172/cs172-assignment1-2023fall-skkk256/output", 0)
    density_maps = []
    for i in range(len(images)):
        density_map = get_density_map_gaussian(images[i][0], gt[i].numpy())
        density_map = cv2.resize(density_map,(128, 128))
        density_maps.append(density_map)
        
    ax0.imshow(np.transpose(images[0], (1, 2, 0)))
    ax0.set_title(str(gt[0].shape[0]))
    ax1.imshow(density_maps[0], cmap=plt.cm.jet)
    ax1.set_title(str(np.sum(density_maps[0]) * 16))
    plt.show()

In [7]:
class L1WithFactor(nn.Module):
    def __init__(self, factor):
        super(L1WithFactor, self).__init__()
        self.factor = factor

    def forward(self, input, target):
        mse = nn.functional.l1_loss(input, target)
        loss = mse * self.factor
        return loss
factor = 100
loss_fn = L1WithFactor(factor)

In [ ]:
model = resnet50()
model

In [13]:
model = MCNN()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = loss_fn

In [14]:
from modules.crowd_counting import Task2_Solver

task2_solver = Task2_Solver(model, optimizer, criterion, model_name="MCNN_B", downsample=True, device=device)

In [15]:
task2_solver.load_checkpoints("/data/shengkuan/CS172/cs172-assignment1-2023fall-skkk256/checkpoints/MCNN_10-29_15-48-42_160.pth")

In [ ]:
epoch = 160
save_per_epoch = 20
log = True

task2_solver.train(epoch, train_loader, log, save_per_epoch=save_per_epoch, val_loader=test_loader)

In [ ]:
task2_solver.test(test_loader, 10, "10-24_20-30")

In [16]:
mae, mse = task2_solver.evaluate_model(test_loader)
print(mae, mse)

tensor(54.9886, device='cuda:2') tensor(163.2789, device='cuda:2')


In [ ]:
print(mae.cpu(), mse.cpu())

In [ ]:
task2_solver(images.to(device))/2000

In [ ]:
dataiter = iter(test_loader)
dataiter = next(dataiter)
images, gt = dataiter

task2_solver.load_checkpoints("/data/shengkuan/CS172/cs172-assignment1-2023fall-skkk256/checkpoints/MCNN_10-29_15-48-42_160.pth")
# task2_solver(images.to(device))
task2_solver.test(train_loader, "test-B", "10-29_18-56")